In [ ]:
pip install NERDA

In [23]:
from collections import defaultdict
import itertools
# import WhitespaceTokenizer() method from nltk
from nltk.tokenize import WhitespaceTokenizer
# Create a reference variable for Class WhitespaceTokenizer
tk = WhitespaceTokenizer()

d = defaultdict(list)
d['sentences'] = []
d['tags'] = []
t = defaultdict(list)
t['sentences'] = []
t['tags'] = []
test = defaultdict(list)
test['sentences'] = []
test['tags'] = []


In [24]:
def nerda_format(data):
  ner = defaultdict(list)
  for line1,line2 in itertools.zip_longest(*[data]*2):
    sents = []
    tags = []
    tokens = iter(tk.tokenize(line1))
    current_tag = "O"
    for token in tokens:
      suffix = ""
      if token in line2:
        suffix = "-V" 
                # Good: even if the verb is complex (aka spaced out),suffix can still apply to the entire phrase
                # Bad: false positive -V may occur, if one word that has multiple meanings occurring more than one time in the same sentence 
      try:
        if token == "[":
          current_tag = "I"
          tags.append("B" + suffix)
          sents.append(next(tokens))
        elif token == "]":
          current_tag = "O"
            #   tags.append(current_tag+suffix)
            #   sents.append(next(tokens))
        else:
          tags.append(current_tag+suffix)
          sents.append(token)
      except StopIteration:
        pass
    ner['sentences'].append(sents)
    ner['tags'].append(tags)
  return ner

In [25]:
with open('dev.txt') as data:
    d = nerda_format(data)
    #d['sentences'] = ner['sentences']
    #d['tags'] = ner['tags']

with open('train.txt') as data:
    t = nerda_format(data)
  #  t['sentences'] = ner['sentences']
  #  t['tags'] = ner['tags']

with open('test.txt') as data:
    test = nerda_format(data)
  #  test['sentences'] = ner['sentences']
  #  test['tags'] = ner['tags']

#print(len(d['sentences']))
#print (d['sentences'][1])
#print (d['tags'][1])

In [26]:
d_total_O = 0
t_total_O = 0
test_total_O = 0

d_total_I = 0
t_total_I = 0
test_total_I = 0

d_total_B = 0
t_total_B = 0
test_total_B = 0

d_total_BV = 0
t_total_BV = 0
test_total_BV = 0

d_total_OV = 0
t_total_OV = 0
test_total_OV = 0

d_total_IV = 0
t_total_IV = 0
test_total_IV = 0


for elem in d['tags']:
  d_total_O += elem.count('O')
  d_total_I += elem.count('I')
  d_total_B += elem.count('B')
  d_total_IV += elem.count('I-V')
  d_total_BV += elem.count('B-V')
  d_total_OV += elem.count('O-V')

for elem in t['tags']:
  t_total_O += elem.count('O')
  t_total_I += elem.count('I')
  t_total_B += elem.count('B')
  t_total_IV += elem.count('I-V')
  t_total_BV += elem.count('B-V')
  t_total_OV += elem.count('O-V')

for elem in test['tags']:
  test_total_O += elem.count('O')
  test_total_I += elem.count('I')
  test_total_B += elem.count('B')
  test_total_IV += elem.count('I-V')
  test_total_BV += elem.count('B-V')
  test_total_OV += elem.count('O-V')



In [27]:
print ('I' + '=' + str(d_total_I + t_total_I + test_total_I))
print ('O' + '=' + str(d_total_O + t_total_O + test_total_O))
print ('B' + '=' + str(d_total_B + t_total_B + test_total_B))
print ('IV' + '=' + str(d_total_IV + t_total_IV + test_total_IV))
print ('OV' + '=' + str(d_total_OV + t_total_OV + test_total_OV))
print ('BV' + '=' + str(d_total_BV + t_total_BV + test_total_BV))

I=2133
O=1618
B=207
IV=199
OV=315
BV=1


In [28]:
import NERDA
training = d
validation = t

In [29]:
print(training.get('sentences')[0])

['अले', 'ग्वल', 'लागाय्', 'नं', 'उगु', 'हे', 'कथं', 'दुःख', 'बिउ', 'वइ', 'धकाः', 'तायेका', 'ग्वलया', 'मनूतय्सं', 'जिमिथाय्', 'मस्त', 'काः', 'वलकि', 'थथे', 'याये', 'धकाः', 'ख्यायेगु', 'नितिं', 'उगु', 'कथं', 'मचायात', 'खड्गय्', 'तिया', 'जात्रा', 'यायेगु', 'यानातःगु', 'खः', 'धाइ']


In [30]:
tag_scheme = [
'B',
'B-V',
'I',
'I-V',
'O-V'
]

In [31]:
from NERDA.models import NERDA
model = NERDA(
dataset_training = training,
dataset_validation = validation,
tag_scheme = tag_scheme, 
tag_outside = 'O',
transformer = 'bert-base-multilingual-cased',
hyperparameters = {'epochs' : 11,
                  'warmup_steps' : 10,
                  'train_batch_size': 5,
                  'learning_rate': 0.0001})

Device automatically set to: cuda


Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.87M [00:00<?, ?B/s]

In [32]:
model.train()


 Epoch 1 / 11


  0%|          | 0/9 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #6 length 327 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
 11%|█         | 1/9 [00:00<00:05,  1.41it/s]/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #16 length 132 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
  0%|          | 0/20 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #6 length 261 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #14 length 152 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
 20%|██        | 4/20 [00:00<00:02,  7.92it/s]/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #48 length 150 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
/

Train Loss = 1.4340456459257338 Valid Loss = 1.1502516746520997

 Epoch 2 / 11


  0%|          | 0/9 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #6 length 327 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
 11%|█         | 1/9 [00:00<00:03,  2.50it/s]/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #16 length 132 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
  0%|          | 0/20 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #6 length 261 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #14 length 152 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
 20%|██        | 4/20 [00:00<00:02,  7.76it/s]/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #48 length 150 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
/

Train Loss = 1.1503707965215046 Valid Loss = 1.1852732360363007

 Epoch 3 / 11


...

  0%|          | 0/9 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #6 length 327 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
 11%|█         | 1/9 [00:00<00:03,  2.45it/s]/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #16 length 132 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
  0%|          | 0/20 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #6 length 261 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #14 length 152 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
 20%|██        | 4/20 [00:00<00:02,  7.66it/s]/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #48 length 150 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
/

Train Loss = 0.2719167371590932 Valid Loss = 0.9550425037741661

 Epoch 11 / 11


  0%|          | 0/9 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #6 length 327 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
 11%|█         | 1/9 [00:00<00:03,  2.47it/s]/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #16 length 132 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
  0%|          | 0/20 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #6 length 261 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #14 length 152 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
 20%|██        | 4/20 [00:00<00:02,  7.71it/s]/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #48 length 150 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
/

Train Loss = 0.24595262441370222 Valid Loss = 0.9581746384501457


'Model trained successfully'

In [33]:
model.evaluate_performance(test)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,Level,F1-Score,Precision,Recall
0,B,0.857143,0.800000,0.923077
1,B-V,0.000000,0.000000,0.000000
2,I,0.848249,0.801471,0.900826
3,I-V,0.173913,0.285714,0.125000
4,O-V,0.384615,0.500000,0.312500
0,AVG_MICRO,0.766467,NaN,NaN
0,AVG_MICRO,0.452784,NaN,NaN


In [37]:
#text= [['अझ', 'वृहद', 'कथं', 'धायेगु', 'खःसा', 'थ्व', 'विज्ञान', 'ब्रह्माण्ड', 'गथे', 'सनाच्वंगु', 'दु', 'धका', 'स्वयेत','याइगु', 'प्रकृतिया', 'साधारण', 'एनालाइसिस', 'ख']]
print(model.predict([test['sentences'][1]]))
print(test['tags'][1])

[['B', 'B', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']]
['B', 'I', 'I', 'I', 'I', 'I-V', 'I', 'I', 'I-V', 'I-V', 'I', 'I', 'O-V', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O-V', 'O-V', 'O']


In [35]:
## Getting the confusion matrices for every test sentence

from sklearn.metrics import confusion_matrix
y_true = test['tags']
for n in range (len(test['tags'])):
  y_pred = []
  y_pred = model.predict([test['sentences'][n]])[0]
  #print(y_true[n])
  #print(y_pred)
  print(confusion_matrix(y_true[n], y_pred, labels=["I", "B", "O","I-V","B-V","O-V"]))

[[ 3  0  0  0  0  0]
 [ 0  1  0  0  0  0]
 [10  0  9  1  0  0]
 [ 1  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  1  0  0]]
[[7 1 0 0 0 0]
 [0 1 0 0 0 0]
 [0 0 8 0 0 0]
 [3 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 3 0 0 0]]
[[4 0 2 0 0 0]
 [0 1 0 0 0 0]
 [0 0 1 0 0 0]
 [1 0 1 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 1]]
[[6 0 1 1 0 0]
 [0 1 0 0 0 0]
 [0 0 9 0 0 1]
 [1 0 0 1 0 0]
 [0 0 0 0 0 0]
 [0 0 1 0 0 0]]
[[10  0  3  0  0  0]
 [ 0  1  0  0  0  0]
 [ 0  0 14  0  0  0]
 [ 1  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 1  0  1  0  0  0]]
[[4 0 0 0 0 0]
 [1 0 0 0 0 0]
 [1 2 0 0 0 1]
 [1 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 1]]
[[4 0 0 1 0 0]
 [0 1 0 0 0 0]
 [0 0 7 0 0 0]
 [1 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 1 0 0 1]]
[[9 0 0 0 0 0]
 [0 1 0 0 0 0]
 [0 0 1 0 0 0]
 [1 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 1 0 0 0]]
[[13  0  0  0  0  0]
 [ 0  1  0  0  0  0]
 [ 0  0  3  0  0  1]
 [ 1  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  1  0  0  1]]
[[9 0 0 0 0 0]
 [0 1 0 0 0 0]
 [0 0 1 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0

In [38]:
# Sum up the 6by6 confusion matrix result in every test sentence
import numpy as np
from sklearn.metrics import confusion_matrix
y_true = test['tags']
d = np.zeros((6,6))
for n in range (len(test['tags'])):
  y_pred = []
  y_pred = model.predict([test['sentences'][n]])[0]
  #print(y_true[n])
  #print(y_pred)
  d += confusion_matrix(y_true[n], y_pred, labels=["I", "B", "O","I-V","B-V","O-V"])
print(d)

[[109.   1.   7.   3.   0.   1.]
 [  1.  12.   0.   0.   0.   0.]
 [ 11.   2.  58.   1.   0.   4.]
 [ 13.   0.   1.   2.   0.   0.]
 [  0.   0.   0.   0.   0.   0.]
 [  2.   0.   8.   1.   0.   5.]]
